In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [6]:
!pip install simpletransformers

In [7]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [8]:
import string
from nltk.corpus import stopwords

def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word.lower() for word in nopunc.split() if word.lower() not in STOPWORDS])

def preprocess(col):
    clean = col.apply(text_process)
    hashtags = col.apply(lambda x: [word.lower() for word in x.split() if word.startswith("#")])
    return clean.to_list(), hashtags.to_list()

In [9]:
train['clean'], train['hashtags']=preprocess(train.text)
test['clean'], test['hashtags']=preprocess(test.text)

In [10]:
train.head()

In [11]:
type(train.clean)

In [12]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model = ClassificationModel('albert', 'albert-base-v2')

In [13]:
model.args

In [14]:
args = {'learning_rate': 2e-05,
               'overwrite_output_dir': True, 
               'num_train_epochs': 5} 

In [15]:
X_train, X_val, y_train, y_val = train_test_split(train['clean'], train['target'], test_size=0.2, random_state=42)
train_data = pd.DataFrame({'clean':X_train, 'target':y_train})
val_data = pd.DataFrame({'clean':X_val, 'target':y_val})
print(train_data.head())

In [16]:
model = ClassificationModel('albert', 'albert-base-v2', args=args)
model.train_model(train_data)
result, outputs, wrong_predictions = model.eval_model(val_data, acc=sklearn.metrics.accuracy_score)
print(result['acc'])

# Train on full

In [17]:
train_data = train[['clean', 'target']]

In [18]:
model = ClassificationModel('albert', 'albert-base-v2', args=args) 
model.train_model(train_data)

# Submit

In [19]:
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
predictions, raw_outputs = model.predict(test_data['text'].to_list())

sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = predictions
sample_submission.to_csv("submission.csv", index=False)